In [6]:
import urllib.request

url = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task"
file_path = "pose_landmarker.task"

print("Đang tải file...")
urllib.request.urlretrieve(url, file_path)
print("Tải xong!")


Đang tải file...
Tải xong!


In [8]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import tensorflow as tf
import time

# Load model đã train
model = tf.keras.models.load_model("lstm_pose_model.h5")

# Cấu hình PoseLandmarker cho nhiều người
base_options = mp.tasks.BaseOptions(model_asset_path="pose_landmarker.task")
options = mp.tasks.vision.PoseLandmarkerOptions(
    base_options=base_options,
    num_poses=2  # Nhận diện tối đa 2 người
)
detector = mp.tasks.vision.PoseLandmarker.create_from_options(options)

# Biến global
labels = {}  # Lưu nhãn của từng người theo ID
n_time_steps = 10  # Số khung hình đầu vào
lm_dict = {}  # Lưu danh sách khung xương theo từng người
warmup_frames = 30  # Số frame chờ trước khi bắt đầu dự đoán
cap = cv2.VideoCapture(1)

# Tạo danh sách class hành động
classes = ["DUNG YEN", "CHAY", "DI BO", "NGOI", "DAM", "VAY TAY"]

# Biến đo FPS
prev_time = 0

# 🏎 Hàm trích xuất khung xương từ MediaPipe
def extract_landmarks(pose_landmarks):
    return [coord for lm in pose_landmarks for coord in (lm.x, lm.y, lm.z, lm.visibility)]

# 🏎 Vẽ khung xương lên ảnh
def draw_landmarks(img, detection_result):
    for i, pose_landmarks in enumerate(detection_result.pose_landmarks):
        pose_proto = mp.framework.formats.landmark_pb2.NormalizedLandmarkList()
        pose_proto.landmark.extend([
            mp.framework.formats.landmark_pb2.NormalizedLandmark(x=lm.x, y=lm.y, z=lm.z)
            for lm in pose_landmarks
        ])
        mp.solutions.drawing_utils.draw_landmarks(
            img, pose_proto, mp.solutions.pose.POSE_CONNECTIONS
        )
    return img

# 🏎 Vẽ nhãn hành động lên ảnh
def draw_labels(img, labels):
    for i, (pid, label) in enumerate(labels.items()):
        cv2.putText(img, f"Person {pid}: {label}", (10, 40 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    return img

# 🏎 Hàm dự đoán hành động
def detect_action(person_id, lm_list):
    global labels
    lm_array = np.array(lm_list).reshape(1, n_time_steps, -1)
    results = model.predict(lm_array)
    predicted_class = np.argmax(results)
    labels[person_id] = classes[predicted_class]

# Vòng lặp chính
frame_count = 0
while True:
    success, img = cap.read()
    if not success:
        break

    # Chuyển ảnh về RGB
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=imgRGB)
    detection_result = detector.detect(mp_image)


    # Đo FPS
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time) if prev_time != 0 else 0
    prev_time = curr_time

    if frame_count > warmup_frames and detection_result.pose_landmarks:
        for i, pose_landmarks in enumerate(detection_result.pose_landmarks):
            lm = extract_landmarks(pose_landmarks)

            # Lưu dữ liệu theo từng người
            if i not in lm_dict:
                lm_dict[i] = []
            lm_dict[i].append(lm)

            # Khi đủ 10 frame, bắt đầu dự đoán
            if len(lm_dict[i]) == n_time_steps:
                t = threading.Thread(target=detect_action, args=(i, lm_dict[i],))
                t.start()
                lm_dict[i] = []  # Reset danh sách

    # Vẽ khung xương và nhãn
    img = draw_landmarks(img, detection_result)
    img = draw_labels(img, labels)

    # Hiển thị FPS
    cv2.putText(img, f"FPS: {int(fps)}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Hiển thị ảnh
    cv2.imshow("Multi-Person Pose Classification", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()


AttributeError: module 'mediapipe' has no attribute 'framework'

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import threading
import time

# Load model đã train
model = tf.keras.models.load_model("lstm_pose_model.h5")

# Biên dịch lại model để tránh cảnh báo
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Tạo PoseLandmarker cho nhiều người
base_options = mp.tasks.BaseOptions(model_asset_path="pose_landmarker.task")
options = mp.tasks.vision.PoseLandmarkerOptions(
    base_options=base_options,
    num_poses=5  # Số người tối đa trong ảnh
)
detector = mp.tasks.vision.PoseLandmarker.create_from_options(options)

# Danh sách hành động
classes = ["DUNG YEN", "CHAY", "DI BO", "NGOI", "DAM", "VAY TAY"]

# Số frame đầu vào
n_time_steps = 10
warmup_frames = 30
cap = cv2.VideoCapture(1)

# Biến lưu khung xương của từng người
people_landmarks = {}
people_labels = {}

# 🏎 Hàm trích xuất khung xương từ PoseLandmarker
def extract_landmarks(detection_result):
    if not detection_result or not detection_result.pose_landmarks:
        return []  # Trả về danh sách rỗng nếu không có người nào trong khung hình

    people_landmarks = []
    for pose_landmarks in detection_result.pose_landmarks:
        if pose_landmarks:  # Kiểm tra nếu không None
            lm = [coord for lm in pose_landmarks for coord in (lm.x, lm.y, lm.z, lm.visibility)]
            people_landmarks.append(lm)
    return people_landmarks

# 🏎 Hàm dự đoán hành động
def detect_action(person_id, lm_list):
    global people_labels
    lm_array = np.array(lm_list).reshape(1, n_time_steps, -1)
    results = model.predict(lm_array)
    predicted_class = np.argmax(results)
    people_labels[person_id] = classes[predicted_class]

# 🏎 Vẽ nhãn cho từng người
def draw_label(img, label, x, y):
    cv2.putText(img, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Vòng lặp chính
frame_count = 0
prev_time = 0
while True:
    success, img = cap.read()
    if not success:
        break

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=imgRGB)
    
    detection_result = detector.detect(mp_image)
    
    # Kiểm tra detection_result có dữ liệu hay không
    if not detection_result or not detection_result.pose_landmarks:
        cv2.putText(img, "Không nhận diện được người!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Pose Classification", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue  # Bỏ qua vòng lặp nếu không có dữ liệu

    # Tính FPS
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time) if prev_time != 0 else 0
    prev_time = curr_time

    if frame_count > warmup_frames:
        extracted_landmarks = extract_landmarks(detection_result)

        for person_id, lm in enumerate(extracted_landmarks):
            if person_id not in people_landmarks:
                people_landmarks[person_id] = []
                people_labels[person_id] = "Đang nhận diện..."

            people_landmarks[person_id].append(lm)

            if len(people_landmarks[person_id]) == n_time_steps:
                t = threading.Thread(target=detect_action, args=(person_id, people_landmarks[person_id],))
                t.start()
                people_landmarks[person_id] = []

            # Vẽ nhãn tại vị trí hông trái
            try:
                left_hip = detection_result.pose_landmarks[person_id][mp.solutions.pose.PoseLandmark.LEFT_HIP]
                x = int(left_hip.x * img.shape[1])
                y = int(left_hip.y * img.shape[0])
                draw_label(img, people_labels.get(person_id, "Đang nhận diện..."), x, y - 20)
            except IndexError:
                pass  # Tránh lỗi nếu không có đủ điểm landmark

    # Hiển thị FPS
    cv2.putText(img, f"FPS: {int(fps)}", (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Hiển thị ảnh
    cv2.imshow("Pose Classification", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
